In [ ]:
using Revise
using HyperbolicDiffEq
using OrdinaryDiffEq, DiffEqCallbacks
using LaTeXStrings, Plots; pyplot()

xmin, xmax = -2., 2.
N = 10^3
parallel = Val{:serial}()

balance_law = Burgers()
fnum = LocalLaxFriedrichsFlux()
reconstruction = ModifiedENO(2)

# set up a periodic problem
u_ana = solve(RiemannProblem(balance_law, 1.e-4, 2., -0.5) * RiemannProblem(balance_law, 2., 1.e-4, 0.5))
tspan = (0., 1.)
u₀func(x) = u_ana(tspan[1], x)

# create an ODE
mesh = UniformPeriodicMesh1D(xmin, xmax, N)
fv = UniformPeriodicReconstructedFV1D(balance_law, mesh, fnum, reconstruction, parallel)
ode = semidiscretise(fv, u₀func, tspan)

# solve the ODE sing either an adaptive time step (CFL) or a fixed one
#cb = StepsizeLimiter((t,u)->max_dt(t,u,fv), safety_factor=1, max_step=true)
sol = solve(ode, OrdinaryDiffEq.Euler(), dt=max_dt(tspan[1], ode.u0, fv), save_everystep=false)


plot(xguide=L"x", yguide=L"u")
plot!(evaluate_coefficients(sol[1], mesh), label=L"u_0")
plot!(evaluate_coefficients(sol[end], mesh), label=L"u_\mathrm{num}")
#x = linspace(xmin, xmax, 10^3)
#plot!(x, u_ana.(tspan[end], x), label=L"u_\mathrm{ana}", legend=true)
